# Tennis Results and Odds Scrapper

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from bs4 import BeautifulSoup
import time

from requests import get
from requests_html import HTMLSession

In [54]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("C:/Users/lorys/Documents/Tennis-Betting/operadriver.exe", chrome_options=options)

<ipython-input-54-e95c8b72335e>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("C:/Users/lorys/Documents/Tennis-Betting/operadriver.exe", chrome_options=options)


### Initializing Variables

In [4]:
starting_url =  'http://www.oddsportal.com/results/#tennis'
starting_url = starting_url.rstrip()

tournament_tours = ['atp','itf','wta','challenger']
#tournament_tours = ['atp']

### Get Response

In [5]:
starting_response = get (starting_url, headers={'User-Agent': 'Mozilla/5.0'})
starting_soup = BeautifulSoup(starting_response.text, 'html.parser')
type(starting_soup)

bs4.BeautifulSoup

In [6]:
tournament_links = starting_soup.find_all(href = re.compile('^\/tennis\/\w+\/.+\/results\/$'))
print(len(tournament_links))
print(tournament_links[0])

2982
<a foo="f" href="/tennis/algeria/itf-tlemcen-women/results/">ITF Tlemcen Women</a>


### Tournaments DataFrame

In [7]:
df_tournaments = pd.DataFrame(columns = ['Name','Country', 'Tour', 'Gender', 'Single/Doubles', 'Link'])
for i in range (len(tournament_links)):
    link = tournament_links[i]['href']
    name = tournament_links[i].text
    
    is_chosen_tour = any(t in link for t in tournament_tours)
    if is_chosen_tour is True:
        
        cutted_link = link.split('/')
        country = cutted_link[2]
        tour = next((t for t in tournament_tours if t in link))
        
        gender = 'Unknown'
        if (tour == 'atp'):
            gender = 'Men'
        elif(tour == 'wta'):
            gender = 'Women'
        else:
            gender = next((g for g in ('women','men') if g in link),'Unknown')
        
        sd = 'Singles'
        if ('doubles' in link):
            sd = 'Doubles'

        
        
        
        df_tournaments= df_tournaments.append(
            {
                'Name': name,
                'Country': country ,
                'Tour': tour,
                'Gender': gender,
                'Single/Doubles': sd ,
                'Link': link
            }
                                          
                                          
                                          
                                          ,ignore_index=True)

### Get All Matches

In [55]:
df_matches = pd.DataFrame(columns = ['Name','Country', 'Tour', 'Gender', 'Single/Doubles', 'Link','Terrain','Year','FullDate','P1','P2','Odd1','Odd2','Score','Winner'])

In [84]:

for i in range (2693,len(df_tournaments)): 
    domain = 'http://www.oddsportal.com'
    link = df_tournaments.loc[i,'Link']
    total_link = domain + link
    

    tournament_mainpage = get (total_link, headers={'User-Agent': 'Mozilla/5.0'})
    tournament_mainpage_soup = BeautifulSoup(tournament_mainpage.text, 'html.parser')
    
    if tournament_mainpage.status_code == 404:
        continue
        
    years = list((l.text for l in tournament_mainpage_soup.find('div', attrs={'class' : 'main-menu2 main-menu-gray'}).find_all('a')))
    page_links = list((l['href'] for l in tournament_mainpage_soup.find('div', attrs={'class' : 'main-menu2 main-menu-gray'}).find_all('a')))
    
    
    print("Progress : ",str(i+1),"/",len(df_tournaments), "  - ", df_tournaments.loc[i,'Name'], "  - ",len(years),"  Years")

    for j in range (len(page_links)):
        page_link = domain + page_links[j]
        driver.get(page_link)
        year_mainpage = driver.page_source
        year_mainpage_soup = BeautifulSoup(year_mainpage, 'html.parser')
        
        try:
            pagination = year_mainpage_soup.find('div',attrs={'id' : 'tournamentTable'}).find('div', attrs={'id' : 'pagination'}).find_all('a')
            nb_page_links = (l['href'] for l in pagination)
            nb_page_links = list(dict.fromkeys(nb_page_links))
        except:
            nb_page_links = [page_links[j]]
            
        
        for k in range (len(nb_page_links)):
            final_link = page_link + nb_page_links[k]
            driver.get(final_link)
            
            
            print(final_link)
            if final_link == 'http://www.oddsportal.com//results///results/':
                print("False Link")
                continue
            
            try:
                final_page = driver.page_source
                final_soup = BeautifulSoup(final_page, 'html.parser')
                table = final_soup.find('div', attrs={'id' : 'tournamentTable'}).find('tbody').find_all('tr')
            except:
                time.sleep(2)
                final_page = driver.page_source
                final_soup = BeautifulSoup(final_page, 'html.parser')
                table = final_soup.find('div', attrs={'id' : 'tournamentTable'}).find('tbody').find_all('tr')
                
            

            try:                
                terrain_type = final_soup.find('h1').text.split('(')[1].split(')')[0]
            except:
                if(j>=1 and terrain_type is not None):
                    pass
                else:
                    terrain_type = 'unknown'
                    
            fulldate = 'unknown'
            
            try:
                #'Noattrs={'class': 'cms','text' : 'No data available'}
                no_data= final_soup.find('div', class_ = 'cms', text = 'No data available')
                if no_data is not None:
                    print('No Data')
                    break
            except:
                pass
                
            for row in table:
                p1 = 'unknown'
                p2 ='unknown'
                odd1 = 'unknown'
                odd2 = 'unknown'
                score ='unknown'
                winner = 'unknown'


                if  'nob-border' in row['class'] :
                    fulldate = row.find('span').text
                
                if 'Today' in fulldate:
                    continue
                    
                elif 'deactivate' in row['class']:
                    players = row.find('td', attrs={'class' : 'table-participant'}).find('a').text
                    p1 = players.split('-')[0].strip()
                    p2 = players.split('-')[1].strip()
                    
                    
                    try:
                        score =  row.find('td', attrs={'class' : 'table-score'}).text
                        nb_set1 = int(score.split(':')[0])
                        nb_set2 = int(score.split(':')[1])
                        if nb_set1>nb_set2:
                            winner = 1
                        elif nb_set1<nb_set2:
                            winner = 2
                        else:
                            winner= 'error'
                            print(winner)
                    except:
                        winner = 'Cancelled'
                    
                    
                    try:
                        odds = row.find_all('td', attrs={'class' : 'odds-nowrp'})
                        odd1 = float(odds[0].text)
                        odd2 = float(odds[1].text)
                    except:
                        pass
                        
                    
                    df_matches= df_matches.append(
                        {
                            'Name': df_tournaments.loc[i,'Name'],
                            'Country': df_tournaments.loc[i,'Country'] ,
                            'Tour': df_tournaments.loc[i,'Tour'],
                            'Gender': df_tournaments.loc[i,'Gender'],
                            'Single/Doubles': df_tournaments.loc[i,'Single/Doubles'] ,
                            'Link': final_link,
                            'Terrain': terrain_type,
                            'Year': years[j],
                            'FullDate': fulldate,
                            'P1': p1,
                            'P2': p2,
                            'Score': score,
                            'Winner':winner,
                            'Odd1':odd1,
                            'Odd2':odd2
                        } , ignore_index=True)
    print ('Nb Match : ', len(df_matches))

df_matches

Progress :  2694 / 2819   -  Indian Wells Challenger Women Doubles   -  3   Years
http://www.oddsportal.com/tennis/usa/indian-wells-challenger-women-doubles/results//tennis/usa/indian-wells-challenger-women-doubles/results/
http://www.oddsportal.com/tennis/usa/indian-wells-challenger-women-doubles-2019/results//tennis/usa/indian-wells-challenger-women-doubles-2019/results/
http://www.oddsportal.com/tennis/usa/indian-wells-challenger-women-doubles-2018/results//tennis/usa/indian-wells-challenger-women-doubles-2018/results/
Nb Match :  227372
Progress :  2695 / 2819   -  ITF Atlanta Women   -  1   Years
http://www.oddsportal.com/tennis/usa/itf-atlanta-women/results//tennis/usa/itf-atlanta-women/results/
Nb Match :  227403
Progress :  2696 / 2819   -  ITF Berkeley Men   -  1   Years
http://www.oddsportal.com/tennis/usa/itf-berkeley-men/results//tennis/usa/itf-berkeley-men/results/
Nb Match :  227434
Progress :  2697 / 2819   -  ITF Boca Raton Men   -  1   Years
http://www.oddsportal.com/t

,Name,Country,Tour,Gender,Single/Doubles,Link,Terrain,Year,FullDate,P1,P2,Odd1,Odd2,Score,Winner
0,ITF Tlemcen Women,algeria,itf,women,Singles,http://www.oddsportal.com/tennis/algeria/itf-t...,clay,2012,14 Apr 2012,Kovinic D.,Romanova A.,unknown,unknown,2:0,1
1,ITF Tlemcen Women,algeria,itf,women,Singles,http://www.oddsportal.com/tennis/algeria/itf-t...,clay,2012,12 Apr 2012,Hyoty J.,Romanova A.,3.75,1.25,0:2,2
2,ITF Tlemcen Women,algeria,itf,women,Singles,http://www.oddsportal.com/tennis/algeria/itf-t...,clay,2012,12 Apr 2012,Kovinic D.,Neuwirth Y.,1.3,3.4,2:0,1
3,ITF Tlemcen Women,algeria,itf,women,Singles,http://www.oddsportal.com/tennis/algeria/itf-t...,clay,2012,11 Apr 2012,Hyoty J.,Wessel A.,3.75,1.25,2:0,1
4,ITF Tlemcen Women,algeria,itf,women,Singles,http://www.oddsportal.com/tennis/algeria/itf-t...,clay,2012,11 Apr 2012,Neuwirth Y.,Gervais F.,1.36,3,2:0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240376,WTA Billie Jean King Cup - Group III Doubles,world,wta,Women,Doubles,http://www.oddsportal.com/tennis/world/wta-fed...,unknown,2017,14 Jun 2017,Grammatikopoulou V./Kordolaimi E.,Genovese E./Pellicano H.,1.07,7.29,canc.,Cancelled
240377,WTA Billie Jean King Cup - Group III Doubles,world,wta,Women,Doubles,http://www.oddsportal.com/tennis/world/wta-fed...,unknown,2017,14 Jun 2017,Hristovska N./Marinkovikj K.,Bechri C./Jabeur O.,10.62,1.02,canc.,Cancelled
240378,WTA Billie Jean King Cup - Group III Doubles,world,wta,Women,Doubles,http://www.oddsportal.com/tennis/world/wta-fed...,unknown,2017,13 Jun 2017,Copas R./Fennelly J.,Brynjarsdottir H./Gronholm A. S.,1.1,6.47,2:0,1
240379,WTA Billie Jean King Cup - Group III Doubles,world,wta,Women,Doubles,http://www.oddsportal.com/tennis/world/wta-fed...,unknown,2017,13 Jun 2017,Omirou E./Siopacha M.,Karanja K./Nkatha J.,1.01,13,2:0,1


In [83]:
df_matches = df_matches[df_matches['Name']!='Indian Wells Challenger Women']

In [85]:
df_matches.to_pickle("./all_matches.pkl")

In [64]:
players

'\xa0'

In [65]:
row

<tr class="odd deactivate" xeid="KtcyOgeF"><td class="table-time datet t1605787200-1-1-0-0">12:00</td><td class="name table-participant" colspan="3"><a class="ico-tv-tournament" href="javascript:void(0);" id="tv-KtcyOgeF" onmouseout="allowHideTootip(true);delayHideTip(2000);" onmouseover="toolTip('&lt;a class=&quot;name-white&quot; href=&quot;/bookmaker/bet365/stream/&quot; onclick=&quot;return !window.open(this.href)&quot; title=&quot;Watch live streaming&quot;&gt;bet365  Live Streaming&lt;/a&gt;&lt;br /&gt;&lt;a class=&quot;name-white&quot; href=&quot;/bookmaker/bwin/stream/&quot; onclick=&quot;return !window.open(this.href)&quot; title=&quot;Watch live streaming&quot;&gt;bwin  Live Streaming&lt;/a&gt;&lt;br /&gt;', this, event, 6);allowHideTootip(false);delayHideTip(2000);return false;" style=""> </a><a href="/tennis/greece/itf-m15-heraklion-5-men-doubles/alves-mateus-rodrigues-natan-brouwer-gijs-veldheer-mick-KtcyOgeF/">Alves M./Rodrigues N. - Brouwer G./Veldheer M.</a></td><td cla